   # Hello FLARE API

In this notebook, we go through the different commands of the FLARE API to show the syntax and usage of each.

### 1. Install NVIDIA FLARE and Provision an FL System

For this notebook, we will need a running NVFLARE project that we can connect to. Follow the [Installation](https://nvflare.readthedocs.io/en/main/getting_started.html#installation) instructions to set up an environment that has NVIDIA FLARE installed if you do not have one already.

Feel free to use an existing **provisioned** NVFLARE project if you have that available.



### 2. Provision the FL System

Since we are using production system, we need to provision first. You can look at the [Provision in NVFLARE](https://nvflare.readthedocs.io/en/latest/programming_guide/provisioning_system.html) and reference the [provision notebook](./provision.ipynb)
Here we will recap some of the commands

We are going to use ```/tmp/workspace``` as provision workspace

You can change the following cells from markdown to code and experiment in each step. 
or just use the scripts 

* tu_pre_start.sh -- to provision and start FL system
* tu_post_cleanup.sh -- to stop and cleanup FL system
 



### 3. Start the FL System

Once you are in the terminal, make sure you are in the ```nvflare_example``` venv. you can setup this by 

```../set_env.sh```


Use 

```./tu_pre_start.sh``` 

to start the FL system in local host for Non-HA, secure mode. If you running the scripts successfully, you should be able to see the output like below

```
  < ...skip output ...>
  
2023-03-31 18:54:57,887 - FederatedClient - INFO - Got engine after 0.5073049068450928 seconds
2023-03-31 18:54:57,887 - FederatedClient - INFO - Got engine after 0.5075399875640869 seconds
2023-03-31 18:54:57,887 - FederatedClient - INFO - Got the new primary SP: grpc://localhost:8002
2023-03-31 18:54:57,887 - FederatedClient - INFO - Got the new primary SP: grpc://localhost:8002

trying to connect to server
Server info:
status: stopped, start_time: Fri Mar 31 18:54:56 2023

Client info
site-1(last_connect_time: Fri Mar 31 18:55:07 2023)
site-2(last_connect_time: Fri Mar 31 18:55:07 2023)
ready to go

```

If you see this, **```ready to go```**, you are ready to go back to notebook and run the job. 

If the you getting errors, **avoid repeatedly run ./tu_pre_start.sh**, first you need to try to shutdown NVFLARE system, using

```
  ./tu_post_cleanup.sh 
```

you can check if the nvflare system are shutdown cleanly. 

```
     ps -eaf | grep nvflare
     
```
If you seen the followings, then the nvflare systems are still running

```

510535    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-1/startup/.. -s fed_client.json --set secure_train=true uid=site-1 org=nvidia config_folder=config
510539    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-2/startup/.. -s fed_client.json --set secure_train=true uid=site-2 org=nvidia config_folder=config
510543    1932  1 18:54 pts/1    00:00:04 python3 -u -m nvflare.private.fed.app.server.server_train -m /tmp/workspace/example_project/prod_00/localhost/startup/.. -s fed_server.json --set secure_train=true org=nvidia config_folder=config

```
make sure they are cleared before you try to start the nvflare again.kill the process if needed. 

Once the NVFLARE system is up and running, then we are ready to check FL system status and runn jobs.



### 4. Connect to the FL System with the FLARE API and Get System Info

Use `new_secure_session()` to initiate a session connecting to the FL Server with the FLARE API. The necessary arguments are the username of the admin user you are using and the corresponding startup kit location (if you are using POC mode, you will need to use `new_insecure_session()` with the startup kit location as the only argument).

In the code example below, we get the `admin_user_dir` by concatenating the workspace root with the default directories that are created if you provision a project with a given project name. The assumption is that 'nvflare provision' has been run in a workspace set to '/workspace' to set up a project named `hello-example` with a server and two clients. You can change the workspace location and the project name to what applies for your environment if needed.

Note that if debug mode is not enabled, there is no output after initiating a session successfully, so instead we print the output of `get_system_info()`.

The `get_system_info()` command does not take any arguments, and returns a SystemInfo object consisting of server_info (server status and start time), client_info (each connected client and the last connect time for that client), and job_info (the list of current jobs with the job_id and app_name).

If you are unable to connect and initiate a session, make sure that your FL Server is running and that the configurations are correct with the right path to the admin startup kit directory.

But first, we need to find out the startup kit parant directory, since we have 
prod_00, prod_01, etc. the following function will just do that and assign the directory to ```provision_dir``` 

In [ ]:
import os
def get_last_provision_dir(project_dir:str): 
    dir_path = project_dir  
    dirs = [os.path.join(dir_path, d) for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d)) and d.startswith("prod_")]
    
    dirs_sorted = sorted(dirs, key=os.path.getctime)
    
    return dirs_sorted[-1]

provision_dir = get_last_provision_dir('/tmp/workspace/example_project')
print(provision_dir)

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_secure_session
 
username = "admin@nvidia.com"
admin_user_dir = os.path.join(provision_dir,username)
sess = new_secure_session(
    username=username,
    startup_kit_location=admin_user_dir
)
print(sess.get_system_info())

### 5. Prepare Examples

We are going to use hello-numpy-sag examples. we need to copy the example to the place jobs are expected. 



In [ ]:
! cp -r  ../hello-world/hello-numpy-sag {provision_dir}/{username}/transfer/.

In [ ]:
! ls -al {provision_dir}/{username}/transfer/

### 6. Submit Job
With a session successfully connected, you can use `submit_job()` to submit your job. You can change `path_to_example_job` to the location of the job you are submitting. Upon successful submission, the job_id is returned as a string.

In [ ]:
path_to_example_job = "hello-numpy-sag/jobs/hello-numpy-sag"
job_id = sess.submit_job(path_to_example_job)
print(job_id + " was submitted")

### 7. Monitor Job

The command `monitor_job()` allows you to follow a job until the job is done.

By default, `monitor_job()` only has one required arguement, the `job_id` of the job you are waiting for, and the default behavior is to wait until the job is complete before returning a Return Code of `JOB_FINISHED`.

In order to follow along and see a more meaningful result and demonstrate some of the possibilities of using this function, the following cell contains a `sample_cb()` callback that keeps track of the number of times the callback is run and prints the `job_meta` the first three times and the final time before `monitor_job()` completes with every other call just printing a dot to save output space. This callback is just an example of what can be done with additional arguments and the `job_meta` information of the job that is being monitored. You can use logic to return a value other than True to conditionally stop `monitor_job()` and return `MonitorReturnCode.ENDED_BY_CB`.

In [ ]:
from nvflare.fuel.flare_api.flare_api import Session

def sample_cb(
        session: Session, job_id: str, job_meta, *cb_args, **cb_kwargs
    ) -> bool:
    if job_meta["status"] == "RUNNING":
        if cb_kwargs["cb_run_counter"]["count"] < 3:
            print(job_meta)
            print(cb_kwargs["cb_run_counter"])
        else:
            print(".", end="")
    else:
        print("\n" + str(job_meta))
    
    cb_kwargs["cb_run_counter"]["count"] += 1
    return True

sess.monitor_job(job_id, cb=sample_cb, cb_run_counter={"count":0})

### 8. Get Job Meta

To get the job meta information, you can use the `get_job_meta()` command. The only argument required for this command is the job id for the job you are getting the meta for. The job meta information will be returned as an object.

In [ ]:
sess.get_job_meta(job_id)

### 9. List Jobs

To get the information for what jobs have been submitted to the server, you can use the `list_jobs()` command. If no arguments are included, the defaults are False for both "detailed" and "reverse".

Setting "detailed" to True will return more detailed information about each job.

Setting "reverse" to True will return the jobs in reverse order by submission time.

The arg "limit" can be set to specify the maximum number of jobs to return, with 0 or None meaning return all jobs (the default is None to show all).

The args "id_prefix" and "name_prefix" can be used to further filter the jobs returned to have an id or name beginning with the string set for the respective argument.

In [ ]:
import json

def format_json( data: dict): 
    print(json.dumps(data, sort_keys=True, indent=4,separators=(',', ': ')))

list_jobs_output = sess.list_jobs()
print(format_json(list_jobs_output))


In [ ]:

list_jobs_output_detailed = sess.list_jobs(detailed=True)
print(format_json(list_jobs_output_detailed))

### 10. Download Job Result

The `download_job_result()` command downloads the job result to the "download_dir" (this is usually set in fed_admin.json in the startup directory of the admin_user_dir used when launching the FLARE API Session, and this value is relative to the admin_user_dir). This command only has one required arguement, the `job_id` of the job result to download.

In [ ]:
sess.download_job_result(job_id)

### 11. Clone Jobs

To clone an existing job, you can use the `clone_job()` command. The `clone_job()` command only has one required arguement, the `job_id` of the job to clone.

In [ ]:
sess.clone_job(job_id)

### 12. Abort Job

If training gets stuck or there is another reason to stop a running job, you can use the `abort_job()` command. The `abort_job()` command only has one required arguement, the `job_id` of the job to abort, and None is returned upon successfully issuing the command.

In [ ]:
print(job_id)
sess.abort_job(job_id)

### 13. Delete Job

You should be able to see the output in the terminals where you are running your FL Server and Clients when you submitted the job. You can also use `monitor_job()` to follow along and give you updates on the progress until the job is done.

By default, `delete_job()` only has one required argument, the `job_id` of the job you are waiting for, and the default behavior is to return nothing if the command is successful. The command will raise a `JobNotDone` exception if the job is still running. 

In [ ]:
sess.delete_job(job_id)

### 14. All Other Commands

For all other commands that do not have a specific FLARE API implementation, using ``api.do_command()`` should provide what the FLAdminAPI would have returned in the raw result.

In [ ]:
sess.api.do_command("list_sp")

### 15. Stop and Clean up FL System
One can use 

```tu_post_cleanup.sh``` 

script to stop FL system and cleanup
